In [1]:
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark import SparkConf, SparkContext

# Start the spark context
sc = SparkContext.getOrCreate(conf=swan_spark_conf)
# create a spark session (which will run spark jobs)
spark = SparkSession.builder.getOrCreate()

In [108]:
import pandas as pd
crash = pd.read_csv("../../Project 1/car_crashes.csv", low_memory=False)

In [110]:
crash = crash.loc[crash["LATITUDE"]>0]
len(crash)/9

176776.66666666666

In [75]:
float(crash.iloc[0,5])


-73.97617

In [14]:
import fiona
import shapely
from shapely.geometry import shape, Point

shapefile = fiona.open("fixed_taxi_zones.shp")
r = shapefile.next()
shape = shapely.geometry.asShape( r['geometry'] )
print(shape.centroid)

POINT (-74.17400027276294 40.69183120640148)


<ipython-input-14-0edd40fe8c6a>:6: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  r = shapefile.next()


In [101]:
#Changing car crash data to have zone instead of lat/lon points
import shapely.speedups
shapely.speedups.enable()

import fiona
from shapely.geometry import shape, Point

shapefile = fiona.open("fixed_taxi_zones.shp")

for i in range(1054):
    for j in range(len(shapefile)):
        shapefile_record = shapefile[j]
        zone_id = shapefile_record['properties']['LocationID']

    # Use Shapely to create the polygon
        shape = shapely.geometry.asShape( shapefile_record['geometry'] )

        checkpoint = shapely.geometry.Point(float(crash.iloc[i, 5]), float(crash.iloc[i,4])) # longitude, latitude

        if shape.contains(checkpoint):
            print("Found shape for point" + str(point) + "at zone " + str(zone_id))
            break

Found shape for point(-73.97617, 40.68358)at zone 97
Found shape for point(-73.97617, 40.68358)at zone 65
Found shape for point(-73.97617, 40.68358)at zone 183
Found shape for point(-73.97617, 40.68358)at zone 33
Found shape for point(-73.97617, 40.68358)at zone 91
Found shape for point(-73.97617, 40.68358)at zone 84
Found shape for point(-73.97617, 40.68358)at zone 242
Found shape for point(-73.97617, 40.68358)at zone 38
Found shape for point(-73.97617, 40.68358)at zone 185
Found shape for point(-73.97617, 40.68358)at zone 36
Found shape for point(-73.97617, 40.68358)at zone 191
Found shape for point(-73.97617, 40.68358)at zone 126
Found shape for point(-73.97617, 40.68358)at zone 204
Found shape for point(-73.97617, 40.68358)at zone 76
Found shape for point(-73.97617, 40.68358)at zone 81
Found shape for point(-73.97617, 40.68358)at zone 39
Found shape for point(-73.97617, 40.68358)at zone 238
Found shape for point(-73.97617, 40.68358)at zone 211
Found shape for point(-73.97617, 40.68

KeyboardInterrupt: 

In [100]:
a = fiona.open("fixed_taxi_zones.shp")
a[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('OBJECTID', 1),
              ('Shape_Leng', 0.116357453189),
              ('Shape_Area', 0.0007823067885),
              ('zone', 'Newark Airport'),
              ('LocationID', 1),
              ('borough', 'EWR')]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(-74.18445299999996, 40.694996000000096),
    (-74.18448899999999, 40.695095000000066),
    (-74.18449799999996, 40.69518500000006),
    (-74.18438099999997, 40.69587800000008),
    (-74.18428199999994, 40.6962110000001),
    (-74.18402099999997, 40.69707500000007),
    (-74.18391299999996, 40.69750700000005),
    (-74.18375099999997, 40.69779500000007),
    (-74.18363399999998, 40.6983260000001),
    (-74.18356199999994, 40.69845200000007),
    (-74.18354399999998, 40.698560000000086),
    (-74.18350799999996, 40.69870400000011),
    (-74.18327399999998, 40.700090000000074),
    (-74.18315699999994, 40.70121500000007),
    (-74.18316599999997, 40.70238500000

In [56]:
import geopandas as gpd
sf = gpd.read_file("../../Project 1/taxi_zones/taxi_zones.shp")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf.to_file("fixed_taxi_zones.shp")

In [40]:
#setting schema
import pyspark.sql.functions as F

from pyspark.sql.types import *
from pyspark.sql.functions import col

spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)

ints = ('VendorID', 'passenger_count', 'RateCodeID', 'RatecodeID','payment_type',)
doubles = ('trip_distance', 'fare_amount', 'extra', 'mta_tax', 'pickup_longitude', 'pickup_latitude', 
           'dropoff_longitude', 'dropoff_latitude','tip_amount', 'tolls_amount', 
           'improvement_surcharge', 'total_amount',)
strings = ('store_and_fwd_flag',)
dtimes = ('tpep_pickup_datetime', 'tpep_dropoff_datetime',)

dtypes = {column: IntegerType() for column in ints}
dtypes.update({column: DoubleType() for column in doubles})
dtypes.update({column: StringType() for column in strings})
dtypes.update({column: TimestampType() for column in dtimes})

schema = StructType()

for column in sdf.columns:
    schema.add(column, # column name
               dtypes[column], # data type
               True # is nullable?
              )

NameError: name 'spark' is not defined

In [60]:
sdf = spark.read.csv('../../MAST30034_Python/data/2015', header=True, schema=schema) \
    .withColumnRenamed("RatecodeID","RateCodeID") # rename the wrong column

sdf.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [61]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
sdf.limit(10)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2015-05-05 23:37:40,2015-05-05 23:45:41,1,2.0,-74.00167846679688,40.73931121826172,1,N,-73.9782943725586,40.75210952758789,2,8.5,0.5,0.5,0.0,0.0,0.3,9.8
2,2015-05-05 23:37:40,2015-05-05 23:40:36,1,0.54,-73.93083953857422,40.744789123535156,1,N,-73.93751525878906,40.749359130859375,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8
2,2015-05-05 23:37:40,2015-05-05 23:44:03,3,2.1,-74.00141143798828,40.73108673095703,1,N,-73.98167419433594,40.75828170776367,2,8.0,0.5,0.5,0.0,0.0,0.3,9.3
2,2015-05-05 23:37:40,2015-05-06 00:14:01,6,10.93,-73.97067260742188,40.75856018066406,1,N,-73.93376159667969,40.6705436706543,1,36.0,0.5,0.5,9.32,0.0,0.3,46.62
2,2015-05-05 23:37:40,2015-05-05 23:46:03,5,0.93,-73.98673248291016,40.75587844848633,1,N,-73.99095916748047,40.74998092651367,1,7.0,0.5,0.5,2.49,0.0,0.3,10.79
1,2015-05-05 23:37:41,2015-05-05 23:50:34,3,2.4,-73.98932647705078,40.75659942626953,1,N,-73.97958374023438,40.7353630065918,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3
1,2015-05-05 23:37:42,2015-05-05 23:41:51,1,1.3,-73.9555435180664,40.7767219543457,1,N,-73.94133758544922,40.78812026977539,2,6.0,0.5,0.5,0.0,0.0,0.3,7.3
2,2015-05-05 23:37:42,2015-05-05 23:53:25,1,3.7,-73.9922103881836,40.72918701171875,1,N,-73.96039581298828,40.775630950927734,1,15.0,0.5,0.5,3.26,0.0,0.3,19.56
2,2015-05-05 23:37:42,2015-05-05 23:52:53,2,7.84,-73.98773956298828,40.74085998535156,1,N,-73.8731689453125,40.72309112548828,1,23.5,0.5,0.5,6.07,5.54,0.3,36.41
2,2015-05-05 23:37:42,2015-05-05 23:48:25,3,2.36,-73.98799896240234,40.743995666503906,1,N,-73.99852752685547,40.717002868652344,2,10.0,0.5,0.5,0.0,0.0,0.3,11.3


In [92]:
#Dropping irrelevant columns
#Rationale is to only have the data available to a user before their trip - e.g. don't know exact trip dist, rate_code ID etc.
sdf = sdf.drop("RateCodeID").drop("store_and_fwd_flag").drop("VendorID").drop("payment_type")\
    .drop("tip_amount").drop("total_amount")
sdf = sdf.withColumnRenamed('tpep_pickup_datetime', 'pickup_time')\
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_time')

#Create cost column --> total amount minus tip amount, don't want to try predict the tip
sdf = sdf.withColumn("cost", col("fare_amount") + col("extra") + col("mta_tax") + col("tolls_amount") + col("improvement_surcharge"))

#Drop other cost columns
#sdf = sdf.drop("fare_amount").drop("extra").drop("mta_tax").drop("tolls_amount").drop("improvement_surcharge")

#Create lat/lon pair for pickup and dropoff, to ease computation of approx distance
from pyspark.sql.functions import struct

sdf = sdf.withColumn("pickup_coord", struct(sdf.pickup_longitude, sdf.pickup_latitude))\
    .withColumn("dropoff_coord", struct(sdf.dropoff_longitude, sdf.dropoff_latitude))

sdf.limit(20)

pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,cost,pickup_coord,dropoff_coord,approx_distance
2015-05-05 23:37:40,2015-05-05 23:45:41,1,2.0,-74.00167846679688,40.73931121826172,-73.9782943725586,40.75210952758789,8.5,0.5,0.5,0.0,0.3,9.8,{-74.001678466796...,{-73.978294372558...,1.6339881624135753
2015-05-05 23:37:40,2015-05-05 23:40:36,1,0.54,-73.93083953857422,40.744789123535156,-73.93751525878906,40.749359130859375,4.5,0.5,0.5,0.0,0.3,5.8,{-73.930839538574...,{-73.937515258789...,0.4694519190253022
2015-05-05 23:37:40,2015-05-05 23:44:03,3,2.1,-74.00141143798828,40.73108673095703,-73.98167419433594,40.75828170776367,8.0,0.5,0.5,0.0,0.3,9.3,{-74.001411437988...,{-73.981674194335...,1.4588440445768487
2015-05-05 23:37:40,2015-05-06 00:14:01,6,10.93,-73.97067260742188,40.75856018066406,-73.93376159667969,40.6705436706543,36.0,0.5,0.5,0.0,0.3,37.3,{-73.970672607421...,{-73.933761596679...,3.0545402962157318
2015-05-05 23:37:40,2015-05-05 23:46:03,5,0.93,-73.98673248291016,40.75587844848633,-73.99095916748047,40.74998092651367,7.0,0.5,0.5,0.0,0.3,8.3,{-73.986732482910...,{-73.990959167480...,0.31291679750135426
2015-05-05 23:37:41,2015-05-05 23:50:34,3,2.4,-73.98932647705078,40.75659942626953,-73.97958374023438,40.7353630065918,11.0,0.5,0.5,0.0,0.3,12.3,{-73.989326477050...,{-73.979583740234...,0.7855086251591178
2015-05-05 23:37:42,2015-05-05 23:41:51,1,1.3,-73.9555435180664,40.7767219543457,-73.94133758544922,40.78812026977539,6.0,0.5,0.5,0.0,0.3,7.3,{-73.955543518066...,{-73.941337585449...,1.005400104310164
2015-05-05 23:37:42,2015-05-05 23:53:25,1,3.7,-73.9922103881836,40.72918701171875,-73.96039581298828,40.775630950927734,15.0,0.5,0.5,0.0,0.3,16.3,{-73.992210388183...,{-73.960395812988...,2.3699337492187142
2015-05-05 23:37:42,2015-05-05 23:52:53,2,7.84,-73.98773956298828,40.74085998535156,-73.8731689453125,40.72309112548828,23.5,0.5,0.5,5.54,0.3,30.34,{-73.987739562988...,{-73.873168945312...,7.923353440564057
2015-05-05 23:37:42,2015-05-05 23:48:25,3,2.36,-73.98799896240234,40.743995666503906,-73.99852752685547,40.717002868652344,10.0,0.5,0.5,0.0,0.3,11.3,{-73.987998962402...,{-73.998527526855...,0.8908830375499545


In [93]:
#Approximate trip distance --> user should know their pickup and dropoff points (could be improved with actual
# gps distance)
import geopy.distance
from pyspark.sql.functions import udf
import math

def distance(origin, destination):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_mi : float

    Examples
    --------
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d * 0.621371


distance_udf = udf(distance)

coord1 = (-74.00167846679688, 40.73931121826172)
coord2 = (-73.9782943725586, 40.75210952758789)

distance(coord1, coord2)

1.6339881624135753

In [94]:
sdf = sdf.withColumn("approx_distance", distance_udf(col("pickup_coord"), col("dropoff_coord")))

In [96]:
#Drop invalid trips

sdf = sdf.filter((sdf.pickup_latitude != 0) & (sdf.pickup_longitude != 0) & \
                 (sdf.dropoff_latitude != 0) & (sdf.dropoff_longitude != 0) &\
                (sdf.cost > 0))

In [97]:
sdf.limit(20)

pickup_time,dropoff_time,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,extra,mta_tax,tolls_amount,improvement_surcharge,cost,pickup_coord,dropoff_coord,approx_distance
2015-05-05 23:37:40,2015-05-05 23:45:41,1,2.0,-74.00167846679688,40.73931121826172,-73.9782943725586,40.75210952758789,8.5,0.5,0.5,0.0,0.3,9.8,{-74.001678466796...,{-73.978294372558...,1.6339881624135753
2015-05-05 23:37:40,2015-05-05 23:40:36,1,0.54,-73.93083953857422,40.744789123535156,-73.93751525878906,40.749359130859375,4.5,0.5,0.5,0.0,0.3,5.8,{-73.930839538574...,{-73.937515258789...,0.4694519190253022
2015-05-05 23:37:40,2015-05-05 23:44:03,3,2.1,-74.00141143798828,40.73108673095703,-73.98167419433594,40.75828170776367,8.0,0.5,0.5,0.0,0.3,9.3,{-74.001411437988...,{-73.981674194335...,1.4588440445768487
2015-05-05 23:37:40,2015-05-06 00:14:01,6,10.93,-73.97067260742188,40.75856018066406,-73.93376159667969,40.6705436706543,36.0,0.5,0.5,0.0,0.3,37.3,{-73.970672607421...,{-73.933761596679...,3.0545402962157318
2015-05-05 23:37:40,2015-05-05 23:46:03,5,0.93,-73.98673248291016,40.75587844848633,-73.99095916748047,40.74998092651367,7.0,0.5,0.5,0.0,0.3,8.3,{-73.986732482910...,{-73.990959167480...,0.31291679750135426
2015-05-05 23:37:41,2015-05-05 23:50:34,3,2.4,-73.98932647705078,40.75659942626953,-73.97958374023438,40.7353630065918,11.0,0.5,0.5,0.0,0.3,12.3,{-73.989326477050...,{-73.979583740234...,0.7855086251591178
2015-05-05 23:37:42,2015-05-05 23:41:51,1,1.3,-73.9555435180664,40.7767219543457,-73.94133758544922,40.78812026977539,6.0,0.5,0.5,0.0,0.3,7.3,{-73.955543518066...,{-73.941337585449...,1.005400104310164
2015-05-05 23:37:42,2015-05-05 23:53:25,1,3.7,-73.9922103881836,40.72918701171875,-73.96039581298828,40.775630950927734,15.0,0.5,0.5,0.0,0.3,16.3,{-73.992210388183...,{-73.960395812988...,2.3699337492187142
2015-05-05 23:37:42,2015-05-05 23:52:53,2,7.84,-73.98773956298828,40.74085998535156,-73.8731689453125,40.72309112548828,23.5,0.5,0.5,5.54,0.3,30.34,{-73.987739562988...,{-73.873168945312...,7.923353440564057
2015-05-05 23:37:42,2015-05-05 23:48:25,3,2.36,-73.98799896240234,40.743995666503906,-73.99852752685547,40.717002868652344,10.0,0.5,0.5,0.0,0.3,11.3,{-73.987998962402...,{-73.998527526855...,0.8908830375499545


In [100]:
sdf.write.format("parquet").save("preprocessed.parquet")

KeyboardInterrupt: 

In [83]:

sample = sdf.limit(100)
sample.toPandas().to_csv("test.csv")

In [12]:
sdf_with_schema.agg(mean("total_amount"))

avg(total_amount)
16.096247288503392


In [16]:
small = sdf_with_schema.sample(False, 0.000001, seed = 0)

In [17]:
small.filter((small.total_amount>30)).count()

11

In [19]:
353/1000

0.353

In [18]:
def brackets(x):
    if x < 7:
        return "Less than $7"
    if x >= 7 and x < 10:
        return "Between $7 and $10"
    if x >= 10 and x < 15:
        return "Between $10 and $15"
    if x >= 15 and x < 20:
        return "Between $15 and $20"
    if x >= 20 and x < 30:
        return "Between $20 and $30"
    if x >= 30:
        return "Greater than $30"
from pyspark.sql.functions import udf
myUDF = udf(brackets)

In [19]:
w_brackets = small.withColumn("price_bracket", myUDF(col("total_amount")))

In [20]:
w_brackets.groupBy('price_bracket').count()

price_bracket,count
Between $15 and $20,16
Greater than $30,11
Between $10 and $15,33
Between $7 and $10,25
Between $20 and $30,15
Less than $7,19


In [21]:
w_brackets.limit(10)

KeyboardInterrupt: 

In [40]:
from pyspark.sql.functions import month, date_format

w_month= small.withColumn("Month", month(col("pickup_time"))).withColumn("DayOfWeek", date_format("pickup_time", "d"))

In [38]:
w_month.groupBy('Month').count()

KeyboardInterrupt: 